# Supervision Integration with DeGirum PySDK (Hailo8L/HAILO8)

This notebook demonstrates using the Supervision library to annotate detection results from DeGirum's PySDK running on Hailo hardware. It follows the same modular style as our official user guides and covers:

- Connecting to local or cloud Hailo inference hosts
- Running object detection using YOLO models
- Visualizing predictions using `supervision` tools (bounding boxes + labels)

In [1]:
import numpy as np
import degirum as dg
import degirum_tools
from supervision.detection.core import Detections
from supervision.draw.color import Color
from supervision.geometry.core import Position
from supervision.annotators.core import LabelAnnotator, BoxAnnotator

In [2]:
def convert_to_supervision_results(results):
    if not results:
        return None, [], [], []
    
    bboxes = np.array([det["bbox"] for det in results], dtype=np.float32)
    class_ids = np.array([det["category_id"] for det in results], dtype=int)
    confidences = np.array([det["score"] for det in results], dtype=np.float32)
    labels = [f'{det["label"]} {det["score"]:.2f}' for det in results]

    detections = Detections(
        xyxy=bboxes,
        class_id=class_ids,
        confidence=confidences
    )
    return detections, bboxes, class_ids, labels

In [ ]:
# Inference & model settings
inference_host_address = "@cloud"
zoo_url = "degirum/hailo"
token = ''
device_type = ["HAILORT/HAILO8L"]
model_name = "yolov8n_relu6_face--640x640_quant_hailort_multidevice_1"

# choose image source
image_source = "../assets/ThreePersons.jpg"

# Initialize Supervision annotators
box_annotator = BoxAnnotator(
    color=Color(r=0, g=255, b=0),
    thickness=2
)
label_annotator = LabelAnnotator(
    color=Color(r=0, g=0, b=0),
    text_color=Color(r=255, g=255, b=255),
    text_position=Position.TOP_LEFT,
    text_scale=0.5,
    text_thickness=1,
    text_padding=4,
    border_radius=2
)

In [ ]:
# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
)
# perform AI model inference on given image source
print(f" Running inference using '{model_name}' on image source '{image_source}'")
inference_result = model(image_source)

In [ ]:
detections, _, _, labels = convert_to_supervision_results(inference_result.results)
annotated = box_annotator.annotate(scene=inference_result.image, detections=detections)
annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

print("Press 'x' or 'q' to stop.")
with degirum_tools.Display("AI Camera") as output_display:
    output_display.show_image(annotated)